In [ ]:
from pathlib import Path
import os
import logging
import pprint
from pprint import pformat
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dynamic_yaml
import yaml

with open('../config/data_config.yaml') as f:
    data = dynamic_yaml.load(f)
    data_cfg = yaml.full_load(dynamic_yaml.dump(data))

logging.basicConfig(level=logging.INFO)
logging.debug(pformat(data_cfg, indent=1, width=100, compact=True))

# preprocess funcctions

In [ ]:
def sort_date(raw_data: pd.DataFrame):
    assert raw_data.index.name == 'Date'
    sort_idx_df = raw_data.sort_index(axis=1)
    sort_pre_df = sort_idx_df.sort_values('Date')
    return sort_pre_df

def remove_na(sort_pre_df:pd.DataFrame, null_values_ratio: float = 0.01):
    assert sort_pre_df.index.is_monotonic_increasing, "`sort_pre_df` has not been sorted."
    if sort_pre_df.isna().sum().sum() == 0:
        logging.info("sort_pre_df don't have nulls.")
        return sort_pre_df
    nulls_max_num = int(len(sort_pre_df)*0.01)
    na_mask = sort_pre_df.isna().sum() < nulls_max_num # null values ratio==1%
    for exceed_null_ratio_idx in np.where(na_mask == False)[0]:
        logging.info(f"exceed_null_ratio item:{sort_pre_df.columns[exceed_null_ratio_idx]}, its number of null_values {sort_pre_df.isna().sum().iloc[exceed_null_ratio_idx]}")
    na_pre_df = sort_pre_df.iloc[::, na_mask.tolist()]
    na_pre_df = na_pre_df.ffill()
    na_pre_df = na_pre_df.bfill()
    assert na_pre_df.isna().sum().sum() == 0, f'Still has {na_pre_df.isna().sum().sum()} null value'

    return na_pre_df

# preprocess setting

In [ ]:
is_save_file = True
raw_file_dir = Path(data_cfg['DIRS']['DATASET_DIR'])/f"raw_data"
preprocessed_file_dir = Path(data_cfg['DIRS']['DATASET_DIR'])/f"is_pre_data"
assert raw_file_dir.exists() and preprocessed_file_dir.exists()

# Preprocess raw dataset

## SP500_20082017

In [ ]:
raw_file_name = 'sp500_hold_19982023_adj_close.csv'
output_file_name = 'sp500_hold_20082017_adj_close-pre.csv'
raw_data = pd.read_csv(raw_file_dir/raw_file_name)
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
raw_data = raw_data.query('Date >= "2008-01-01" and Date <= "2017-12-31"')
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df

## SP500_20082012

In [ ]:
raw_file_name = 'sp500_hold_19982023_adj_close.csv'
output_file_name = 'sp500_hold_20082012_adj_close-pre.csv'
raw_data = pd.read_csv(raw_file_dir/raw_file_name)
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
raw_data = raw_data.query('Date >= "2008-01-01" and Date <= "2012-12-31"')
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df

## SP500_20112015

In [ ]:
raw_file_name = 'sp500_hold_19982023_adj_close.csv'
output_file_name = 'sp500_hold_20112015_adj_close-pre.csv'
raw_data = pd.read_csv(raw_file_dir/raw_file_name)
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
raw_data = raw_data.query('Date >= "2011-01-01" and Date <= "2015-12-31"')
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df

## SP500_20122017

In [ ]:
raw_file_name = 'sp500_hold_19982023_adj_close.csv'
output_file_name = 'sp500_hold_20122017_adj_close-pre.csv'
raw_data = pd.read_csv(raw_file_dir/raw_file_name)
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
raw_data = raw_data.query('Date >= "2012-01-01" and Date <= "2017-12-31"')
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df

## Tetuan City power consumption Dataset

In [ ]:
raw_file_name = 'Tetuan City power consumption.csv'
output_file_name = 'tetuan_city_power_consumption-pre.csv'
raw_data = pd.read_csv(raw_file_dir/raw_file_name)
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
raw_data = raw_data.set_index('Date')
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df.iloc[::18, ::] # extract for every 3 hour

## Nvidia stock & Bitcoin

In [ ]:
bitcoin_raw_file_name = 'bitcoin_20102022.csv'
nvda_raw_file_name = 'nvda_20102022.csv'
output_file_name = "bitcoin_nvda_20122022-pre.csv"
bitcoin_raw_data = pd.read_csv(raw_file_dir/bitcoin_raw_file_name)
nvda_raw_data = pd.read_csv(raw_file_dir/nvda_raw_file_name)
bitcoin_raw_data['Date'] = pd.to_datetime(bitcoin_raw_data['Date'])
nvda_raw_data['Date'] = pd.to_datetime(nvda_raw_data['Date'])
raw_data = pd.merge(bitcoin_raw_data, nvda_raw_data, on=["Date"], how="right").set_index("Date")
display(raw_data)
sort_pre_df = sort_date(raw_data)
na_pre_df = remove_na(sort_pre_df)
output_df = na_pre_df.iloc[len(na_pre_df)-2519:, ::]

# Output Data

In [ ]:
if is_save_file:
    output_df.to_csv(preprocessed_file_dir/output_file_name)
    logging.info(f"{output_file_name} has been saved to {preprocessed_file_dir}")
display(output_df)